# 데이터 전처리

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import matplotlib as mpl
mpl.rc('font', family="Malgun Gothic")

from sklearn.preprocessing import LabelEncoder # for encoding
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler #for standardization
from sklearn.model_selection import train_test_split

In [ ]:
def preprocessing(df_scaling, scaled_form = 'MinMaxScaler()'):
    # 불필요한 컬럼 제거
    if 'ID' in df_scaling:
        df_scaling = df_scaling.drop("ID", axis = 1)
        if len(df_scaling.columns) == 1:
            return df_scaling
        
        df_scaling = df_scaling.drop('oral', axis = 1) 
        # 범주형 피처 레이블 인코딩 
        cate_features = df_scaling[['gender','tartar']]

        lbe = LabelEncoder()
        lbe.fit_transform(df_scaling["gender"])
        df_scaling["gender"] = lbe.fit_transform(df_scaling["gender"])

        lbe = LabelEncoder()
        lbe.fit_transform(df_scaling["tartar"])
        df_scaling["tartar"] = lbe.fit_transform(df_scaling["tartar"])

        # 정규화 작업
        cate_features = df_scaling[['gender', 'tartar', 'hearing(right)', 'hearing(left)', 'dental caries']]
        scaled_features = df_scaling.drop(cate_features.columns, axis=1)
        if scaled_form == 'StandardScaler()':
            # Standard scaler
            scaler = StandardScaler()
            scaler.fit(scaled_features)
            scaled = scaler.transform(scaled_features)
            standard_scaled = pd.DataFrame(scaled,columns = scaled_features.columns)
            standard_scaled[cate_features.columns] = cate_features

            return standard_scaled
        else:
            # MinMax scaler
            scaler_M = MinMaxScaler()
            scaler_M.fit(scaled_features)
            scaled_M = scaler_M.transform(scaled_features)
            min_max_scaled = pd.DataFrame(scaled_M,columns = scaled_features.columns)
            min_max_scaled[cate_features.columns] = cate_features

            return min_max_scaled

In [ ]:
smoking = pd.read_csv('./data/smoking.csv')
smoking.head()

In [ ]:
scaled_data = preprocessing(smoking)
scaled_data.head()

In [ ]:
# import numpy as np
# import pandas as pd
# import matplotlib.pyplot as plt
# import seaborn as sns
# %matplotlib inline

In [ ]:
# smoking = pd.read_csv("./data/smoking.csv")
# df = smoking.drop("ID", axis = 1)
# df.head()

In [ ]:
scaled_data.info()

In [ ]:
target = scaled_data['smoking']
target

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(scaled_data, target, test_size = 0.1, random_state=0)

# 데이터 시각화 및 분석
- 데이터 세트의 성별 및 흡연 비율 시각화 (파이)
- 혈관 및 호흡 / 신장 질환 / 간수치 / 구강건강 별 시각화 (히스토그램)
    - 각 해당 히트맵을 통한 흡연과의 상관성 파악

In [ ]:
# 데이터 세트에서 흡연 비율
scaled_data['smoking'].value_counts().plot.pie(explode=[0,0.1],autopct='%1.1f%%',shadow=True)

# Non-smoking = 63.3%
# Smoking =36.7 %
# 비흡연자 > 흡연자

In [ ]:
# 비흡연자(0)와 흡연자(1)의 성별 비율

ex3 = smoking.groupby('smoking')['gender'].value_counts(normalize=True).unstack()
print(ex3)
ex3.plot.bar(stacked=True,figsize=(8,6))
plt.legend(loc='upper right')
plt.title('Gender')

# 비흡연자 : 여성 > 남성
# 흡연자 : 여성 < 남성

In [ ]:
# 혈관 및 호흡
scaled_data1 = scaled_data[['systolic','relaxation','fasting blood sugar',
                            'Cholesterol','triglyceride','HDL','LDL','hemoglobin','smoking']]
scaled_data1.hist(figsize=(10,10))

In [ ]:
scaled_data1.corr().style.background_gradient(cmap = "magma")
# 흡연과 가장 상관관계가 있는 부분은 hemoglobin 이다.

In [ ]:
sns.boxplot(data=scaled_data,x='smoking',y='hemoglobin')
# 흡연과 hemoglobin의 상관계수가 0.4로 흡연 여부가 1일수록 헤모글로빈이 높아짐을 확인 할 수 있다. 

In [ ]:
sns.boxplot(data=scaled_data,x='smoking',y='triglyceride')
# 흡연과 triglyceride의 상관계수가 0.25로 흡연 여부가 1일수록 triglyceride이 높아짐을 확인 할 수 있다. 

In [ ]:
sns.boxplot(data=scaled_data,x='smoking',y='HDL')
# 흡연과 HDL의 상관계수가 -0.17로 흡연 여부가 1일수록 HDL이 낮아짐을 확인 할 수 있다. 

In [ ]:
# 신장질환
scaled_data2 = scaled_data[['Urine protein','serum creatinine','smoking']]
scaled_data2.hist(figsize=(8,8))

In [ ]:
scaled_data2.corr().style.background_gradient(cmap = "magma")
# Urine protein, serum creatinine 와 smoking 은 큰 상관성을 가지고 있지 않음

In [ ]:
sns.boxplot(data=scaled_data,x='smoking',y='serum creatinine')
# 흡연과 serum creatinine의 상관계수가 0.21로 흡연 여부가 1일수록 serum creatinine이 높아짐을 확인 할 수 있다. 

In [ ]:
# 간수치 
scaled_data3 = scaled_data[['AST','ALT','Gtp','smoking']]
scaled_data3.hist(figsize=(8,8))

In [ ]:
scaled_data3.corr().style.background_gradient(cmap = "magma")
# AST, ALT, Gtp 와 역시 큰 상관성이 없음 / Gtp

In [ ]:
sns.boxplot(data=scaled_data,x='smoking',y='Gtp')
# 흡연과 Gtp의 상관계수가 0.23으로 흡연 여부가 1일수록 Gtp이 높아짐을 확인 할 수 있다. 

In [ ]:
# 구강건강 (1)
smoke_dental = scaled_data.groupby("smoking")["dental caries"].value_counts(normalize=True)
pd.DataFrame(smoke_dental)
# smoke_dental.plot(kind='pie', explode=[0,0.1], autopct='%1.1f%%',shadow=True)
# 흡연자 중 충치가 있을 확률 : 59%
# 비흡연자 중 충치가 있을 확률 : 40% 

In [ ]:
# 비흡연자 / 흡연자의 충치 비율 시각화
ex2=smoking.groupby('smoking')['dental caries'].value_counts(normalize=True).unstack()
ex2.plot.bar(stacked=True,figsize=(8,6))
plt.legend(loc='upper right')
plt.title('Dental Caries')

In [ ]:
# 구강건강 (2)

smoke_tarter = scaled_data.groupby("smoking")['tartar'].value_counts(normalize=True)
pd.DataFrame(smoke_tarter)
# print(smoke_tarter)
# smoke_tarter.plot(kind='pie', explode=[0,0.1], autopct='%1.1f%%',shadow=True)
# # 흡연자 중 치석이 있을 확률 : 54%
# # 비흡연자 중 치석이 있을 확률 : 45% 

In [ ]:
# 비흡연자 / 흡연자의 치석 비율 시각화
ex=smoking.groupby('smoking')['tartar'].value_counts(normalize=True).unstack()
ex.plot.bar(stacked=True,figsize=(8,6))
plt.title('Tartar')